In [1]:
import requests, re, time
from bs4 import BeautifulSoup
import scraping_class
import pprint
import pandas as pd
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)
session = requests.session()
session.headers['wmd776@alumni.ku.dk'] = 'wmd776@alumni.ku.dk' 
session.headers['Amalie Tokkesdal'] = 'Amalie Tokkesdal'
session.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'wmd776@alumni.ku.dk': 'wmd776@alumni.ku.dk', 'Amalie Tokkesdal': 'Amalie Tokkesdal'}

In [2]:
url='https://www.imdb.com/chart/top'
number_of_movies = 250
response = requests.get(url)

if response.ok:
    html = response.text
else:
    print('error')
    
soup = BeautifulSoup(html,'html.parser')

link_location = html.split('href="')[1:]

links = set()


for link_loc in link_location:
    link = link_loc.split('"')[0]
    if '/title/' in link:
        links.add(link) 
links = ['https://www.imdb.com/'+link for link in links if not 'ref' in link]# add the domain to each link

links

['https://www.imdb.com//title/tt0266697/',
 'https://www.imdb.com//title/tt0108052/',
 'https://www.imdb.com//title/tt0469494/',
 'https://www.imdb.com//title/tt0097165/',
 'https://www.imdb.com//title/tt1979320/',
 'https://www.imdb.com//title/tt0367110/',
 'https://www.imdb.com//title/tt0119217/',
 'https://www.imdb.com//title/tt0057012/',
 'https://www.imdb.com//title/tt0075314/',
 'https://www.imdb.com//title/tt2380307/',
 'https://www.imdb.com//title/tt0114709/',
 'https://www.imdb.com//title/tt0076759/',
 'https://www.imdb.com//title/tt0058946/',
 'https://www.imdb.com//title/tt0120382/',
 'https://www.imdb.com//title/tt2106476/',
 'https://www.imdb.com//title/tt0099685/',
 'https://www.imdb.com//title/tt0081505/',
 'https://www.imdb.com//title/tt2267998/',
 'https://www.imdb.com//title/tt0120586/',
 'https://www.imdb.com//title/tt0080678/',
 'https://www.imdb.com//title/tt0264464/',
 'https://www.imdb.com//title/tt0482571/',
 'https://www.imdb.com//title/tt0167261/',
 'https://w

In [67]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))

def page_extractor(url):
    r, call_id = connector.get(url, "LOL")
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        title, year = soup.select("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1")[0].text.split("\xa0")
        budget = find_budget(soup)
        cumulative_worldwide_gross=float(soup.select("#titleDetails > div:nth-child(15)")[0].text.split("$")[1].replace(",", "").rstrip())
        genre
        
    except:
        soup = None
        title = None
        budget = None
        #cumulative_worldwide_gross= None                                 
        print(url, "didnt work")
    return {
        "title": title,
        "year": year,
        "budget": budget,
        "url": url
        #"cumulative_world_gross": cumulative_worldwide_gross
    }

In [68]:
res = []
for idx, i in enumerate(links[:15]):
    print(idx + 1, "of", len(links[:15]))
    res.append(page_extractor(i))

1 of 15
2 of 15
3 of 15
4 of 15
5 of 15
6 of 15
7 of 15
8 of 15
9 of 15
10 of 15
11 of 15
12 of 15
13 of 15
14 of 15
15 of 15


In [69]:
pd.DataFrame(res)

,budget,title,url,year
0,30000000.0,Kill Bill: Vol. 1,https://www.imdb.com//title/tt0266697/,(2003)
1,22000000.0,Schindlers liste,https://www.imdb.com//title/tt0108052/,(1993)
2,25000000.0,There Will Be Blood,https://www.imdb.com//title/tt0469494/,(2007)
3,16400000.0,Døde poeters klub,https://www.imdb.com//title/tt0097165/,(1989)
4,38000000.0,Rush,https://www.imdb.com//title/tt1979320/,(2013)
5,NaN,"Swades: We, the People",https://www.imdb.com//title/tt0367110/,(2004)
6,10000000.0,Good Will Hunting,https://www.imdb.com//title/tt0119217/,(1997)
7,1800000.0,Dr. Strangelove - eller Hvordan jeg lærte at g...,https://www.imdb.com//title/tt0057012/,(1964)
8,1300000.0,Taxi Driver,https://www.imdb.com//title/tt0075314/,(1976)
9,175000000.0,Coco,https://www.imdb.com//title/tt2380307/,(2017)


In [53]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            print(i.text.strip())
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
find_budget(s)

Budget:$22,000,000
            (estimated)


22000000.0

In [24]:
soup#.find("div", {"id": "titleDetails"})


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<style>
                body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
            </style>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb Top 250 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>


In [7]:
soup.select("#titleDetails > div:nth-child(15)")

[]

In [8]:
url_ = links[1]
r = requests.get(url_)

url_

'https://www.imdb.com//title/tt0108052/'

In [9]:
soup_ = BeautifulSoup(r.text,'html.parser')

In [10]:
#soup_.select("#titleDetails > div:nth-child(15)")


#float(soup_.select("#titleDetails > div:nth-child(12)")[0].text.split("$")[1].split("\n")[0].replace(",", "").rstrip())


In [13]:
rating = soup_.find_all('div', class_="ratingValue")

rating_array=[]
for r in rating:
    try:
        rating_1=r.find("span", class_="ratingValue")
        rating_array.append(rating_1.getText())
    except:
        rating_array.append(0)

    
rating_array



[0]